In [1]:
cd ~/bench

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [2]:
import pandas as pd
import polars as pl
from pathlib import Path

In [18]:
"data/source_tables/yadl/movies_large-yadl-depleted.parquet"
"data/source_tables/yadl/movies_vote_large-yadl-depleted.parquet"
    

'data/source_tables/yadl/movies_vote_large-yadl-depleted.parquet'

In [33]:
pl.read_parquet("data/source_tables/yadl/movies_vote_large-yadl-depleted.parquet").shape

(7397, 2)

In [20]:
raw_data_path = Path("data/source_tables/raw")
yadl_data_path = Path("data/source_tables/yadl")
od_data_path = Path("data/source_tables/open_data_us")

# Movies improved

In [21]:
import ast


def clean_genres(ll):
    g = ast.literal_eval(ll)
    try:
        l1 = g[0]["name"]
        return l1
    except IndexError:
        return ""


def clean_production_companies(ll):
    try:
        g = ast.literal_eval(ll)
    except ValueError:
        return ""
    except SyntaxError:
        print(ll)
    try:
        l1 = g[0]["name"]
        return l1
    except IndexError:
        return ""
    except TypeError:
        return ""


def clean_production_country(ll):
    try:
        g = ast.literal_eval(ll)
    except ValueError:
        return ""
    try:
        l1 = g[0]["iso_3166_1"]
        return l1
    except IndexError:
        return ""
    except TypeError:
        return ""


def clean_spoken_language(ll):
    try:
        g = ast.literal_eval(ll)
    except ValueError:
        return ""
    try:
        l1 = g[0]["name"]
        return l1
    except IndexError:
        return ""
    except TypeError:
        return ""

In [22]:
df = pl.read_parquet("data/source_tables/movie_revenues.parquet").to_pandas()

In [23]:
df = df.drop(
    [
        "belongs_to_collection",
        "homepage",
        "imdb_id",
        "overview",
        "tagline",
        "poster_path",
        "release_date",
    ],
    axis=1,
)

df.genres = df.genres.apply(clean_genres)
df.production_companies = df.production_companies.apply(clean_production_companies)
df.production_countries = df.production_countries.apply(clean_production_country)
df.spoken_languages = df.spoken_languages.apply(clean_spoken_language)

In [24]:
df = df.drop(["yago4_col_to_embed", "raw_entities"], axis=1).rename(
    {"yago3_col_to_embed": "col_to_embed"}, axis=1
)

df = pl.from_pandas(df).drop_nulls("col_to_embed").to_pandas()
df["col_to_embed"] = "<" + df["col_to_embed"] + ">"

In [25]:
df.to_parquet(Path(yadl_data_path, "movies_large-yadl.parquet"))

In [30]:
df_yadl = pl.read_parquet(Path(yadl_data_path, "movies_large-yadl.parquet"))

df_yadl.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "movies_large-yadl-depleted.parquet")
)

df_vote = df_yadl.with_columns(target=pl.col("vote_average")).drop("vote_average")
df_vote.write_parquet(Path(yadl_data_path, "movies_vote_large-yadl.parquet"))
df_vote.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "movies_vote_large-yadl-depleted.parquet")
)

df_yadl.select(pl.col("original_title"), pl.col("target")).write_parquet(
    Path(od_data_path, "movies_large-depleted-open_data.parquet")
)

df_vote.select(pl.col("original_title"), pl.col("target")).write_parquet(
    Path(od_data_path, "movies_vote_large-depleted-open_data.parquet")
)

In [31]:
df_vote.shape

(7397, 19)

In [32]:
yadl_data_path

PosixPath('data/source_tables/yadl')

# US Accidents improved

In [35]:
df = pl.read_parquet("data/source_tables/us_accidents.parquet")

In [36]:
df = (
    df.with_columns(
        County=pl.col("raw_entities")
        .str.split(",")
        .list.to_struct()
        .struct.rename_fields(["County", "State"])
    )
    .unnest("County")
    .drop("raw_entities", "yago4_col_to_embed")
    .rename({"yago3_col_to_embed": "col_to_embed"})
    .with_columns(
        col_to_embed="<" + pl.col("col_to_embed") + ">"
    )
)

In [38]:
df_2021 = df.filter(pl.col("Year") == 2021).drop("Year")

In [39]:
df_2021.write_parquet(Path(od_data_path, "us_accidents_2021-open_data.parquet"))
df_2021.write_parquet(Path(yadl_data_path, "us_accidents_2021-yadl.parquet"))
df_2021.select("County", "State","target").write_parquet(Path(od_data_path, "us_accidents_2021-depleted-open_data_County.parquet"))
df_2021.select("col_to_embed","target").write_parquet(Path(yadl_data_path, "us_accidents_2021-yadl-depleted.parquet"))

In [40]:
df.write_parquet(Path(od_data_path, "us_accidents_large-open_data.parquet"))
df.write_parquet(Path(yadl_data_path, "us_accidents_large-yadl.parquet"))
df.select("County", "State","Year","target").write_parquet(Path(od_data_path, "us_accidents_large-depleted-open_data_County.parquet"))
df.select("Year","col_to_embed","target").write_parquet(Path(yadl_data_path, "us_accidents_large-yadl-depleted.parquet"))

# Company employees

In [8]:
df = pl.read_parquet(Path(raw_data_path, "company_employees.parquet"))
df.write_parquet(Path(od_data_path, "company_employees-open_data.parquet"))

In [10]:
df_yadl = pl.read_parquet(Path(yadl_data_path, "company_employees-yadl.parquet"))
df_yadl.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "company_employees-depleted-yadl.parquet")
)

df.select(pl.col("name"), pl.col("target")).write_parquet(
    Path(od_data_path, "company_employees-depleted-open_data.parquet")
)

# Housing prices

In [13]:
df = pl.read_parquet(Path(yadl_data_path, "housing_prices-yadl.parquet"))
df.write_parquet(Path(yadl_data_path, "housing_prices-yadl.parquet"))
df.write_parquet(Path(raw_data_path, "housing_prices-yadl.parquet"))
df.drop("col_to_embed").write_parquet(
    Path(od_data_path, "housing_prices-open_data.parquet")
)

In [14]:
df

RegionID,SizeRank,City,RegionType,StateName,Code,Metro,County,State,target,col_to_embed
i64,i64,str,str,str,str,str,str,str,f64,str
6181,0,"""New York""","""city""","""NY""","""NY""","""New York-Newar…","""Queens County""","""New York""",5.854972,"""<New_York_City…"
17222,43,"""Buffalo""","""city""","""NY""","""NY""","""Buffalo-Cheekt…","""Erie County""","""New York""",5.334042,"""<Buffalo,_New_…"
832063,45,"""Rochester""","""city""","""NY""","""NY""","""Rochester, NY""","""Monroe County""","""New York""",5.322101,"""<Rochester,_Ne…"
34937,145,"""Yonkers""","""city""","""NY""","""NY""","""New York-Newar…","""Westchester Co…","""New York""",5.779882,"""<Yonkers,_New_…"
7353,149,"""Syracuse""","""city""","""NY""","""NY""","""Syracuse, NY""","""Onondaga Count…","""New York""",5.233641,"""<Syracuse,_New…"
40779,229,"""Schenectady""","""city""","""NY""","""NY""","""Albany-Schenec…","""Schenectady Co…","""New York""",5.435997,"""<Schenectady,_…"
37074,246,"""Albany""","""city""","""NY""","""NY""","""Albany-Schenec…","""Albany County""","""New York""",5.455378,"""<Albany,_New_Y…"
34819,698,"""White Plains""","""city""","""NY""","""NY""","""New York-Newar…","""Westchester Co…","""New York""",5.825207,"""<White_Plains,…"
831538,699,"""Binghamton""","""city""","""NY""","""NY""","""Binghamton, NY…","""Broome County""","""New York""",5.200401,"""<Binghamton,_N…"


# US Elections

In [7]:
df = pl.read_parquet(Path(yadl_data_path, "us_elections-yadl.parquet"))
df_dem = df.filter(pl.col("party") == "DEMOCRAT")
df_dem.write_parquet(Path(yadl_data_path, "us_elections_dem-yadl.parquet"))
df_dem.drop("col_to_embed").write_parquet(
    Path(od_data_path, "us_elections_dem-open_data.parquet")
)
df_dem.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "us_elections_dem-yadl-depleted.parquet")
)
df_dem.select(pl.col("target"), pl.col("county_name")).write_parquet(
    Path(od_data_path, "us_elections_dem-open_data-depleted.parquet")
)